In [7]:
import requests
import pandas as pd

r = requests.get(
    "https://www.vivino.com/api/explore/explore",
    params = {
        "country_code": "ES",
        "country_codes[]":"es",
        "currency_code":"EUR",
        "grape_filter":"varietal",
        "min_rating":"1",
        "order_by":"price",
        "order":"asc",
        "page": 2,
        "price_range_max":"500",
        "price_range_min":"0",
        "wine_type_ids[]":"1"
    },
    headers= {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
    }
)
results = [
    (
        t["vintage"]["wine"]["winery"]["name"], 
        f'{t["vintage"]["wine"]["name"]} {t["vintage"]["year"]}',
        t["vintage"]["statistics"]["ratings_average"],
        t["vintage"]["statistics"]["ratings_count"],

    )
    for t in r.json()["explore_vintage"]["matches"]
]
dataframe = pd.DataFrame(results,columns=['Winery','Wine','Rating','num_review'])

In [8]:
dataframe

,Winery,Wine,Rating,num_review
0,Hoya de Cadenas,Merlot 2021,3.2,275
1,J. Fernando,Barrica 12 Meses Tempranillo 2022,3.7,123
2,Hoya de Cadenas,Cabernet Sauvignon Crianza 2021,3.4,2032
3,Ramón Roqueta,Synera Tinto 2023,3.0,1792
4,El Circo,Acróbata Garnacha 2023,3.4,4030
5,El Circo,Contorsionista Cabernet Sauvignon 2023,3.3,1118
6,Bodega Sommos,Glárima Merlot - Tempranillo 2023,3.5,47
7,Osborne,Solaz Tempranillo - Cabernet Sauvignon 2022,3.7,225
8,Félix Solís,Casa Albali Tempranillo - Shiraz 2021,3.6,117
9,Bodegas Ayuso,Estola Crianza 2020,3.5,50


In [9]:
def fetch_wine_reviews(wine_id, vintage_year, page_number):

     # Define the user agent header
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0"
    }

    # Define the API URL with parameters
    vivino_url = f"https://www.vivino.com/api/wines/{wine_id}/reviews?per_page=50&year={vintage_year}&page={page_number}"

    # Fetch data from the API and parse it as JSON
    api_data = requests.get(vivino_url, headers=headers).json()

    return api_data

In [10]:
# Define the column names for the DataFrame
columns = ["Winery", "Year", "Wine ID", "Wine", "Rating", "num_review", "price", 'Country', 'Region']

# Create an empty DataFrame with the specified column names
wine_data_df = pd.DataFrame(relevant_data, columns=columns)

NameError: name 'relevant_data' is not defined

In [14]:
import requests
import pandas as pd

# Lista de países y sus códigos para la API
countries = {
    "España": "es",
    "Portugal": "pt",
    "Francia": "fr",
    "Italia": "it"
}

# Lista para almacenar los resultados de todas las páginas y países
all_results = []

# Configurar la cantidad de páginas a recorrer por cada país
num_pages = 50  # Ajusta según lo necesario

for country_name, country_code in countries.items():
    print(f"Procesando vinos de {country_name}...")
    for page in range(1, num_pages + 1):
        print(f"Procesando página {page} para {country_name}...")
        try:
            # Hacer la solicitud para cada país y página
            r = requests.get(
                "https://www.vivino.com/api/explore/explore",
                params={
                    "country_code": country_code.upper(),
                    "country_codes[]": country_code,
                    "currency_code": "EUR",
                    "grape_filter": "varietal",
                    "min_rating": "1",
                    "order_by": "price",
                    "order": "asc",
                    "page": page,
                    "price_range_max": "500",
                    "price_range_min": "0",
                    "wine_type_ids[]": "1"
                },
                headers={
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
                }
            )

            # Comprobar si la respuesta tiene datos
            matches = r.json().get("explore_vintage", {}).get("matches", [])
            if not matches:
                print(f"No hay más datos para {country_name}.")
                break

            # Extraer los datos de la página actual
            results = [
                (
                    t["vintage"]["wine"]["winery"].get("name", None),  # Bodega
                    t["vintage"]["wine"].get("name", None),  # Nombre del vino
                    t["vintage"]["statistics"].get("ratings_average", None),  # Rating promedio
                    t["vintage"]["statistics"].get("ratings_count", None),  # Número de opiniones
                    t.get("price", {}).get("amount", None),  # Precio
                    country_name,  # País
                    t["vintage"]["wine"]["winery"].get("region", {}).get("name", None),  # Región
                    ", ".join(t["vintage"].get("food", {}).get("pairings", []))  # Pairings (maridajes)
                )
                for t in matches
            ]

            # Agregar los resultados de esta página a la lista general
            all_results.extend(results)

        except Exception as e:
            print(f"Error en la página {page} para {country_name}: {e}")
            break

# Crear un DataFrame con los resultados acumulados
dataframe = pd.DataFrame(
    all_results, columns=['Winery', 'Wine', 'Rating', 'num_reviews', 'Price', 'Country', 'Region', 'Food Pairings']
)

# Verificar el tamaño del DataFrame
print(f"Total de filas obtenidas: {len(dataframe)}")

# Guardar el DataFrame en un archivo CSV (opcional)
dataframe.to_csv("vivino_wines_extended.csv", index=False)

print(dataframe.head())




Procesando vinos de España...
Procesando página 1 para España...
Procesando página 2 para España...
Procesando página 3 para España...
Procesando página 4 para España...
Procesando página 5 para España...
Procesando página 6 para España...
Procesando página 7 para España...
Procesando página 8 para España...
Procesando página 9 para España...
Procesando página 10 para España...
Procesando página 11 para España...
Procesando página 12 para España...
Procesando página 13 para España...
Procesando página 14 para España...
Procesando página 15 para España...
Procesando página 16 para España...
Procesando página 17 para España...
Procesando página 18 para España...
Procesando página 19 para España...
Procesando página 20 para España...
Procesando página 21 para España...
Procesando página 22 para España...
Procesando página 23 para España...
Procesando página 24 para España...
Procesando página 25 para España...
Procesando página 26 para España...
Procesando página 27 para España...
Procesa

In [23]:
dataframe.sample(10)

,Winery,Wine,Rating,num_reviews,Price,Country,Region,Food Pairings
2301,Cartuxa,EA Reserva Tinto,4.0,830,10.28,Portugal,None,
4789,Masi,Bonacosta Valpolicella Classico,3.8,1061,10.20,Italia,None,
1117,Luberri,Biga de Luberri Crianza,3.8,53,9.25,España,None,
2761,Jean-Francois Protheau,Beaujolais-Villages,3.5,42,8.60,Francia,None,
1145,Valtravieso,Finca Santa Maria Roble,3.7,3880,9.40,España,None,
1891,Quinta do Monte d'Oiro,Reserva,4.3,36,37.18,Portugal,None,
680,Bodega Pirineos,Señorío de Lazán Crianza,3.5,2286,7.80,España,None,
3251,Domaine Lafage,Authentique Côtes du Roussillon Rouge,3.9,302,10.90,Francia,None,
3296,Château Mauvesin Barton,L'Impression Moulis en Médoc,3.4,593,10.95,Francia,None,
2091,Quinta Vale D. Maria,Vinhas Velhas,4.4,266,54.95,Portugal,None,


In [24]:
# Imprime un ejemplo de la respuesta JSON para analizar
print(r.json()["explore_vintage"]["matches"][0])


{'vintage': {'id': 177429186, 'seo_name': 'abbazia-di-novacella-stiftskellerei-neustift-schiava-2023', 'name': 'Abbazia di Novacella (Stiftskellerei Neustift) Schiava 2023', 'statistics': {'status': 'Normal', 'ratings_count': 1005, 'ratings_average': 3.8, 'labels_count': 18, 'wine_ratings_count': 1005, 'wine_ratings_average': 3.8, 'wine_status': ''}, 'image': {'location': '//images.vivino.com/thumbs/Ag5lVh1lTii3ejZ9XZsTow_pl_480x640.png', 'variations': {'bottle_large': '//images.vivino.com/thumbs/Ag5lVh1lTii3ejZ9XZsTow_pb_x960.png', 'bottle_medium': '//images.vivino.com/thumbs/Ag5lVh1lTii3ejZ9XZsTow_pb_x600.png', 'bottle_medium_square': '//images.vivino.com/thumbs/Ag5lVh1lTii3ejZ9XZsTow_pb_600x600.png', 'bottle_small': '//images.vivino.com/thumbs/Ag5lVh1lTii3ejZ9XZsTow_pb_x300.png', 'bottle_small_square': '//images.vivino.com/thumbs/Ag5lVh1lTii3ejZ9XZsTow_pb_300x300.png', 'label': '//images.vivino.com/thumbs/Ag5lVh1lTii3ejZ9XZsTow_pl_480x640.png', 'label_large': '//images.vivino.com/th

# Imprime un ejemplo de la respuesta JSON para analizar
print(r.json()["explore_vintage"]["matches"][0])
